In [ ]:
import numpy as np # linear algebra
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50, DenseNet121, MobileNet, MobileNetV2
from tensorflow.keras.optimizers import AdamW
import math
import itertools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50, DenseNet121, MobileNet, MobileNetV2,VGG16
from tensorflow.keras.optimizers import AdamW
import math
import itertools

data_dir = r"/kaggle/input/eye-diseases-classification/dataset"
filepaths = []
labels = []
folds = os.listdir(data_dir)
for fold in folds:
    foldpath = os.path.join(data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis=1)


print("Class distribution:")
print(df['labels'].value_counts())
print(f"Number of unique classes: {df['labels'].nunique()}")

# Split data
train_df, dummy_df = train_test_split(df, train_size=0.8, shuffle=True, random_state=42)
valid_df, test_df = train_test_split(dummy_df, train_size=0.6, shuffle=True, random_state=42)

print(f"Train samples: {len(train_df)}")
print(f"Valid samples: {len(valid_df)}")
print(f"Test samples: {len(test_df)}")

# Create data generators
batch_size = 32
img_size = (224, 224)
channels = 3

tr_gen = ImageDataGenerator(rescale=1./255)
ts_gen = ImageDataGenerator(rescale=1./255)

train_gen = tr_gen.flow_from_dataframe(
    train_df, x_col='filepaths', y_col='labels', 
    target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=True, batch_size=batch_size
)
valid_gen = ts_gen.flow_from_dataframe(
    valid_df, x_col='filepaths', y_col='labels', 
    target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=True, batch_size=batch_size
)
test_gen = ts_gen.flow_from_dataframe(
    test_df, x_col='filepaths', y_col='labels', 
    target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=False, batch_size=batch_size
)

# Get number of classes
num_classes = 4
class_names = list(train_gen.class_indices.keys())
print(f"Classes: {class_names}")

# ViT Small Implementation
class PatchEmbedding(layers.Layer):
    def __init__(self, patch_size=16, embed_dim=384):
        super().__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.projection = layers.Conv2D(
            embed_dim, kernel_size=patch_size, strides=patch_size, padding='valid'
        )
        
    def call(self, x):
        # x shape: (batch_size, height, width, channels)
        batch_size = tf.shape(x)[0]
        patches = self.projection(x)  # (batch_size, num_patches_h, num_patches_w, embed_dim)
        patches = tf.reshape(patches, [batch_size, -1, self.embed_dim])
        return patches

class MultiHeadAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        self.qkv = layers.Dense(embed_dim * 3)
        self.projection = layers.Dense(embed_dim)
        
    def call(self, x):
        batch_size, seq_len, embed_dim = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]
        
        # Generate Q, K, V
        qkv = self.qkv(x)  # (batch_size, seq_len, embed_dim * 3)
        qkv = tf.reshape(qkv, [batch_size, seq_len, 3, self.num_heads, self.head_dim])
        qkv = tf.transpose(qkv, [2, 0, 3, 1, 4])  # (3, batch_size, num_heads, seq_len, head_dim)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        # Attention
        scale = tf.cast(self.head_dim, tf.float32) ** -0.5
        attn = tf.matmul(q, k, transpose_b=True) * scale
        attn = tf.nn.softmax(attn, axis=-1)
        
        out = tf.matmul(attn, v)
        out = tf.transpose(out, [0, 2, 1, 3])  # (batch_size, seq_len, num_heads, head_dim)
        out = tf.reshape(out, [batch_size, seq_len, embed_dim])
        
        return self.projection(out)

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.attn = MultiHeadAttention(embed_dim, num_heads)
        self.dropout1 = layers.Dropout(dropout)
        
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = keras.Sequential([
            layers.Dense(mlp_hidden_dim, activation='gelu'),
            layers.Dropout(dropout),
            layers.Dense(embed_dim),
            layers.Dropout(dropout)
        ])
        
    def call(self, x, training=None):
        # Self attention
        attn_out = self.attn(self.norm1(x))
        x = x + self.dropout1(attn_out, training=training)
        
        # MLP
        mlp_out = self.mlp(self.norm2(x), training=training)
        x = x + mlp_out
        
        return x

class VisionTransformerSmall(keras.Model):
    def __init__(self, 
                 img_size=224,
                 patch_size=16, 
                 num_classes=4,
                 embed_dim=384,
                 depth=12,
                 num_heads=6,
                 mlp_ratio=4.0,
                 dropout=0.1):
        super().__init__()
        
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        
        # Patch embedding
        self.patch_embed = PatchEmbedding(patch_size, embed_dim)
        
        # Class token and position embedding
        self.class_token = self.add_weight(
            shape=(1, 1, embed_dim),
            initializer='random_normal',
            trainable=True,
            name='class_token'
        )
        
        self.pos_embed = self.add_weight(
            shape=(1, self.num_patches + 1, embed_dim),
            initializer='random_normal',
            trainable=True,
            name='pos_embed'
        )
        
        self.dropout = layers.Dropout(dropout)
        
        # Transformer blocks
        self.blocks = [
            TransformerBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(depth)
        ]
        
        self.norm = layers.LayerNormalization(epsilon=1e-6)
        self.head = layers.Dense(num_classes)
        
    def call(self, x, training=None):
        batch_size = tf.shape(x)[0]
        
        # Patch embedding
        x = self.patch_embed(x)  # (batch_size, num_patches, embed_dim)
        
        # Add class token
        class_tokens = tf.broadcast_to(self.class_token, [batch_size, 1, tf.shape(self.class_token)[-1]])
        x = tf.concat([class_tokens, x], axis=1)
        
        # Add position embedding
        x = x + self.pos_embed
        x = self.dropout(x, training=training)
        
        # Transformer blocks
        for block in self.blocks:
            x = block(x, training=training)
            
        x = self.norm(x)
        
        # Classification head (use class token)
        class_token_final = x[:, 0]
        return self.head(class_token_final)

# Create ViT Small model
def create_vit_small(num_classes):
    return VisionTransformerSmall(
        img_size=224,
        patch_size=16,
        num_classes=num_classes,
        embed_dim=384,
        depth=12,
        num_heads=6,
        mlp_ratio=4.0,
        dropout=0.1
    )

# Create comparison models
def create_efficientnet(num_classes):
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def create_resnet50(num_classes):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def create_densenet121(num_classes):
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model
def create_VGG(num_classes):
    base_model=VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(224,224,3)
    )
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model
def create_mobilenet_v1(num_classes):
    base_model = MobileNet(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def create_mobilenet_v2(num_classes):
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Enhanced training history visualization function
def plot_clean_training_history(history, model_name):
    tr_acc = history.history['accuracy']
    tr_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    
    # Find best epochs
    index_loss = np.argmin(val_loss)
    val_lowest = val_loss[index_loss]
    index_acc = np.argmax(val_acc)
    acc_highest = val_acc[index_acc]
    
    Epochs = [i+1 for i in range(len(tr_acc))]
    loss_label = f'best epoch= {str(index_loss + 1)}'
    acc_label = f'best epoch= {str(index_acc + 1)}'
    
    # Plot training history
    plt.figure(figsize=(15, 8))
    plt.style.use('fivethirtyeight')
    
    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, label='Training loss', linewidth=2)
    plt.plot(Epochs, val_loss, label='Validation loss', linewidth=2)
    plt.scatter(index_loss + 1, val_lowest, s=150, c='blue', label=loss_label, zorder=5)
    plt.title(f'{model_name} - Training and Validation Loss', fontsize=14, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label='Training Accuracy', linewidth=2)
    plt.plot(Epochs, val_acc, 'g', label='Validation Accuracy', linewidth=2)
    plt.scatter(index_acc + 1, acc_highest, s=150, c='blue', label=acc_label, zorder=5)
    plt.title(f'{model_name} - Training and Validation Accuracy', fontsize=14, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print best epoch information
    print(f"\n{model_name} Training Summary:")
    print(f"  Best Validation Loss: {val_lowest:.4f} at epoch {index_loss + 1}")
    print(f"  Best Validation Accuracy: {acc_highest:.4f} at epoch {index_acc + 1}")
    print(f"  Final Training Accuracy: {tr_acc[-1]:.4f}")
    print(f"  Final Validation Accuracy: {val_acc[-1]:.4f}")
    print(f"  Overfitting (Train-Val): {tr_acc[-1] - val_acc[-1]:.4f}")

# Combined training history visualization for all models
def plot_combined_clean_histories(histories):
   
    plt.figure(figsize=(20, 12))
    plt.style.use('fivethirtyeight')
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#FF9FF3']
    
    # Training Loss
    plt.subplot(2, 2, 1)
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = [i+1 for i in range(len(history.history['loss']))]
        plt.plot(epochs, history.history['loss'], 
                color=colors[idx % len(colors)], linewidth=2, label=model_name)
    plt.title('Training Loss Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Validation Loss
    plt.subplot(2, 2, 2)
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = [i+1 for i in range(len(history.history['val_loss']))]
        val_loss = history.history['val_loss']
        plt.plot(epochs, val_loss, 
                color=colors[idx % len(colors)], linewidth=2, label=model_name)
        
        # Mark best epoch
        best_epoch = np.argmin(val_loss) + 1
        best_loss = min(val_loss)
        plt.scatter(best_epoch, best_loss, 
                   color=colors[idx % len(colors)], s=100, zorder=5)
        
    plt.title('Validation Loss Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Training Accuracy
    plt.subplot(2, 2, 3)
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = [i+1 for i in range(len(history.history['accuracy']))]
        plt.plot(epochs, history.history['accuracy'], 
                color=colors[idx % len(colors)], linewidth=2, label=model_name)
    plt.title('Training Accuracy Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Validation Accuracy
    plt.subplot(2, 2, 4)
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = [i+1 for i in range(len(history.history['val_accuracy']))]
        val_acc = history.history['val_accuracy']
        plt.plot(epochs, val_acc, 
                color=colors[idx % len(colors)], linewidth=2, label=model_name)
        
        # Mark best epoch
        best_epoch = np.argmax(val_acc) + 1
        best_acc = max(val_acc)
        plt.scatter(best_epoch, best_acc, 
                   color=colors[idx % len(colors)], s=100, zorder=5)
        
    plt.title('Validation Accuracy Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Model compilation and training function
def compile_and_train_model(model, model_name, train_gen, valid_gen, epochs=50):
    print(f"\n{'='*50}")
    print(f"Training {model_name}")
    print(f"{'='*50}")
    
    # Learning rate schedule
    initial_lr = 1e-3 if 'ViT' in model_name else 1e-4
    total_steps = len(train_gen) * epochs
    lr_schedule = keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_lr,
        decay_steps=total_steps,
        alpha=0.1
    )
    
    # Compile model
    optimizer = AdamW(learning_rate=lr_schedule, weight_decay=1e-4) if 'ViT' in model_name else keras.optimizers.Adam(learning_rate=lr_schedule)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy')]
    )
    
    # Train model
    history = model.fit(
        train_gen,
        validation_data=valid_gen,
        epochs=40,
        verbose=1
    )
    
    # Plot individual training history with clean visualization
    plot_clean_training_history(history, model_name)
    
    return model, history

# Inference time measurement
def measure_inference_time(model, test_gen, num_batches=10):
    times = []
    
    for i, (x_batch, y_batch) in enumerate(test_gen):
        if i >= num_batches:
            break
        
        start_time = time.time()
        _ = model.predict(x_batch, verbose=0)
        end_time = time.time()
        
        times.append(end_time - start_time)
    
    avg_time_per_batch = np.mean(times)
    avg_time_per_image = avg_time_per_batch / batch_size
    
    return avg_time_per_batch, avg_time_per_image

# Confusion Matrix Plot
def plot_confusion_matrix(y_true, y_pred, class_names, model_name):
    """Plot confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    plt.style.use('default')  # Use default style for confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()
    
    return cm

# Precision-Recall Curve
def plot_precision_recall_curves(y_true_onehot, y_scores, class_names, model_name):
    """Plot precision-recall curves for all classes"""
    plt.figure(figsize=(12, 8))
    plt.style.use('default')
    
    colors = ['blue', 'red', 'green', 'orange']
    
    for i in range(len(class_names)):
        precision, recall, _ = precision_recall_curve(y_true_onehot[:, i], y_scores[:, i])
        ap_score = average_precision_score(y_true_onehot[:, i], y_scores[:, i])
        
        plt.subplot(2, 2, i+1)
        plt.plot(recall, precision, color=colors[i], linewidth=2)
        plt.fill_between(recall, precision, alpha=0.3, color=colors[i])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'{class_names[i]} (AP: {ap_score:.3f})')
        plt.grid(True, alpha=0.3)
    
    plt.suptitle(f'Precision-Recall Curves - {model_name}', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Calculate mean AP
    mean_ap = np.mean([average_precision_score(y_true_onehot[:, i], y_scores[:, i]) 
                      for i in range(len(class_names))])
    
    return mean_ap

# Create and train models
models_to_compare = {
    'VGG16':create_VGG(num_classes),
    'ViT Small': create_vit_small(num_classes),
    'MobileNet-V1': create_mobilenet_v1(num_classes),
    'MobileNet-V2': create_mobilenet_v2(num_classes)
}

# Store results
results = {}
histories = {}
confusion_matrices = {}
inference_times = {}

# Train each model
for model_name, model in models_to_compare.items():
    print(f"Model: {model_name}")
    
    # Build the model using a real batch from train_gen if not built
    if not model.built:
        # Get one batch of real images from train_gen
        x_batch, y_batch = next(iter(train_gen))
        _ = model(x_batch)  # Build the model by calling it on real data
    
    print(f"Parameters: {model.count_params():,}")
    
    # Train model
    trained_model, history = compile_and_train_model(
        model, model_name, train_gen, valid_gen, epochs=25
    )
    
    # Evaluate on test set
    test_loss, test_acc, test_top2_acc = trained_model.evaluate(test_gen, verbose=0)
    
    # Get predictions for confusion matrix and PR curves
    test_gen.reset()  # Reset generator
    y_pred_probs = trained_model.predict(test_gen, verbose=0)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    
    # Get true labels
    y_true = test_gen.classes
    y_true_onehot = tf.keras.utils.to_categorical(y_true, num_classes)
    
    # Measure inference time
    test_gen.reset()
    avg_batch_time, avg_image_time = measure_inference_time(trained_model, test_gen)
    inference_times[model_name] = {
        'batch_time': avg_batch_time,
        'image_time': avg_image_time
    }
    
    # Plot confusion matrix
    cm = plot_confusion_matrix(y_true, y_pred_classes, class_names, model_name)
    confusion_matrices[model_name] = cm
    
    # Plot precision-recall curves
    mean_ap = plot_precision_recall_curves(y_true_onehot, y_pred_probs, class_names, model_name)
    
    # Calculate additional metrics
    class_report = classification_report(y_true, y_pred_classes, 
                                       target_names=class_names, output_dict=True)
    
    results[model_name] = {
        'test_accuracy': test_acc,
        'test_top2_accuracy': test_top2_acc,
        'test_loss': test_loss,
        'parameters': model.count_params(),
        'best_val_accuracy': max(history.history['val_accuracy']),
        'mean_ap': mean_ap,
        'precision_macro': class_report['macro avg']['precision'],
        'recall_macro': class_report['macro avg']['recall'],
        'f1_macro': class_report['macro avg']['f1-score'],
        'inference_time_per_image': avg_image_time,
        'inference_time_per_batch': avg_batch_time
    }
    
    histories[model_name] = history
    
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Top-2 Accuracy: {test_top2_acc:.4f}")
    print(f"Mean AP: {mean_ap:.4f}")
    print(f"Inference time per image: {avg_image_time*1000:.2f} ms")
    print(f"Best Validation Accuracy: {max(history.history['val_accuracy']):.4f}")

# Plot combined clean training histories
print(f"\n{'='*60}")
print("COMBINED TRAINING HISTORIES VISUALIZATION")
print(f"{'='*60}")
plot_combined_clean_histories(histories)

# Enhanced visualization function for comprehensive results
def plot_comprehensive_results(results, histories, inference_times):
    # Create comprehensive comparison plots
    fig, axes = plt.subplots(3, 3, figsize=(20, 18))
    plt.style.use('default')  # Use default style for comprehensive plots
    
    models = list(results.keys())
    colors = plt.cm.Set3(np.linspace(0, 1, len(models)))
    
    # Test Accuracy Comparison
    test_accs = [results[model]['test_accuracy'] for model in models]
    bars1 = axes[0, 0].bar(models, test_accs, color=colors)
    axes[0, 0].set_title('Test Accuracy Comparison', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_ylim(0, 1)
    axes[0, 0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(test_accs):
        axes[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # Parameters vs Accuracy
    params = [results[model]['parameters'] / 1e6 for model in models]  # in millions
    scatter = axes[0, 1].scatter(params, test_accs, s=100, c=colors, alpha=0.7)
    axes[0, 1].set_title('Parameters vs Test Accuracy', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Parameters (Millions)')
    axes[0, 1].set_ylabel('Test Accuracy')
    for i, model in enumerate(models):
        axes[0, 1].annotate(model, (params[i], test_accs[i]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    # Inference Time Comparison
    inf_times = [results[model]['inference_time_per_image'] * 1000 for model in models]  # ms
    bars2 = axes[0, 2].bar(models, inf_times, color=colors)
    axes[0, 2].set_title('Inference Time per Image', fontsize=14, fontweight='bold')
    axes[0, 2].set_ylabel('Time (ms)')
    axes[0, 2].tick_params(axis='x', rotation=45)
    for i, v in enumerate(inf_times):
        axes[0, 2].text(i, v + max(inf_times)*0.02, f'{v:.1f}', ha='center', va='bottom')
    
    # F1-Score Comparison
    f1_scores = [results[model]['f1_macro'] for model in models]
    bars3 = axes[1, 0].bar(models, f1_scores, color=colors)
    axes[1, 0].set_title('F1-Score (Macro Average)', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('F1-Score')
    axes[1, 0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(f1_scores):
        axes[1, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # Mean Average Precision
    mean_aps = [results[model]['mean_ap'] for model in models]
    bars4 = axes[1, 1].bar(models, mean_aps, color=colors)
    axes[1, 1].set_title('Mean Average Precision', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('mAP')
    axes[1, 1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(mean_aps):
        axes[1, 1].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # Efficiency Score (Accuracy / Parameters)
    efficiency = [results[model]['test_accuracy'] / (results[model]['parameters'] / 1e6) for model in models]
    bars5 = axes[1, 2].bar(models, efficiency, color=colors)
    axes[1, 2].set_title('Efficiency Score (Acc/Params)', fontsize=14, fontweight='bold')
    axes[1, 2].set_ylabel('Efficiency Score')
    axes[1, 2].tick_params(axis='x', rotation=45)
    for i, v in enumerate(efficiency):
        axes[1, 2].text(i, v + max(efficiency)*0.02, f'{v:.3f}', ha='center', va='bottom')
    
    # Training History - Best Validation Accuracy per Model
    best_val_accs = [max(histories[model].history['val_accuracy']) for model in models]
    bars6 = axes[2, 0].bar(models, best_val_accs, color=colors)
    axes[2, 0].set_title('Best Validation Accuracy', fontsize=14, fontweight='bold')
    axes[2, 0].set_ylabel('Best Val Accuracy')
    axes[2, 0].tick_params(axis='x', rotation=45)
    axes[2, 0].set_ylim(0, 1)
    for i, v in enumerate(best_val_accs):
        axes[2, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # Convergence Analysis (Epochs to Best Validation)
    epochs_to_best = [np.argmax(histories[model].history['val_accuracy']) + 1 for model in models]
    bars7 = axes[2, 1].bar(models, epochs_to_best, color=colors)
    axes[2, 1].set_title('Epochs to Best Validation Accuracy', fontsize=14, fontweight='bold')
    axes[2, 1].set_ylabel('Epochs')
    axes[2, 1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(epochs_to_best):
        axes[2, 1].text(i, v + max(epochs_to_best)*0.02, f'{v}', ha='center', va='bottom')
    
    # Speed vs Accuracy Trade-off
    axes[2, 2].scatter(inf_times, test_accs, s=100, c=colors, alpha=0.7)
    axes[2, 2].set_title('Speed vs Accuracy Trade-off', fontsize=14, fontweight='bold')
    axes[2, 2].set_xlabel('Inference Time (ms)')
    axes[2, 2].set_ylabel('Test Accuracy')
    for i, model in enumerate(models):
        axes[2, 2].annotate(model, (inf_times[i], test_accs[i]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    plt.tight_layout()
    plt.show()

# Print detailed results
print("\n" + "="*100)
print("COMPREHENSIVE RESULTS COMPARISON")
print("="*100)

results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('test_accuracy', ascending=False)

print(f"{'Model':<15} {'Test Acc':<10} {'mAP':<8} {'F1':<8} {'Params(M)':<10} {'Time(ms)':<10} {'Efficiency':<10}")
print("-" * 100)

for model_name, row in results_df.iterrows():
    efficiency = row['test_accuracy'] / (row['parameters'] / 1e6)
    print(f"{model_name:<15} "
          f"{row['test_accuracy']:<10.4f} "
          f"{row['mean_ap']:<8.3f} "
          f"{row['f1_macro']:<8.3f} "
          f"{row['parameters']/1e6:<10.2f} "
          f"{row['inference_time_per_image']*1000:<10.1f} "
          f"{efficiency:<10.3f}")

print("\n" + "="*100)
print("DETAILED ANALYSIS:")
print("="*100)

best_model = results_df.index[0]
print(f" Best Overall Model: {best_model}")
print(f"   - Test Accuracy: {results_df.loc[best_model, 'test_accuracy']:.4f}")
print(f"   - Mean AP: {results_df.loc[best_model, 'mean_ap']:.4f}")
print(f"   - Parameters: {results_df.loc[best_model, 'parameters']/1e6:.2f}M")

# Efficiency analysis
efficiency_scores = results_df['test_accuracy'] / (results_df['parameters'] / 1e6)
most_efficient = efficiency_scores.idxmax()
print(f"\n Most Efficient Model: {most_efficient}")
print(f"   - Efficiency Score: {efficiency_scores[most_efficient]:.4f}")
print(f"   - Test Accuracy: {results_df.loc[most_efficient, 'test_accuracy']:.4f}")

# Speed analysis
fastest_model = results_df['inference_time_per_image'].idxmin()
print(f"\n Fastest Model: {fastest_model}")
print(f"   - Inference Time: {results_df.loc[fastest_model, 'inference_time_per_image']*1000:.1f} ms")
print(f"   - Test Accuracy: {results_df.loc[fastest_model, 'test_accuracy']:.4f}")

# Plot comprehensive results
plot_comprehensive_results(results, histories, inference_times)

# Create training summary for all models
def create_training_summary(histories, results):
    """Create a summary table of training characteristics"""
    training_data = []
    
    for model_name, history in histories.items():
        val_acc = history.history['val_accuracy']
        val_loss = history.history['val_loss']
        train_acc = history.history['accuracy']
        
        # Calculate training metrics
        best_epoch_acc = np.argmax(val_acc) + 1
        best_epoch_loss = np.argmin(val_loss) + 1
        best_val_acc = max(val_acc)
        best_val_loss = min(val_loss)
        final_overfitting = train_acc[-1] - val_acc[-1]
        epochs_trained = len(val_acc)
        
        # Learning stability (std dev of last 5 epochs)
        if len(val_acc) >= 5:
            stability = np.std(val_acc[-5:])
        else:
            stability = np.std(val_acc)
        
        training_data.append({
            'Model': model_name,
            'Best Val Acc': f"{best_val_acc:.4f}",
            'Best Epoch (Acc)': best_epoch_acc,
            'Best Epoch (Loss)': best_epoch_loss,
            'Final Overfitting': f"{final_overfitting:.4f}",
            'Epochs Trained': epochs_trained,
            'Stability (σ)': f"{stability:.4f}",
            'Test Accuracy': f"{results[model_name]['test_accuracy']:.4f}"
        })
    
    training_df = pd.DataFrame(training_data)
    training_df = training_df.sort_values('Best Val Acc', ascending=False)
    
    print("\n" + "="*110)
    print("TRAINING CHARACTERISTICS SUMMARY")
    print("="*110)
    print(training_df.to_string(index=False))
    
    return training_df

# Create training summary
training_summary = create_training_summary(histories, results)

# Create final comprehensive summary table
def create_final_summary_table(results):
    """Create final comprehensive summary table"""
    summary_data = []
    
    for model_name, metrics in results.items():
        summary_data.append({
            'Model': model_name,
            'Test Accuracy': f"{metrics['test_accuracy']:.4f}",
            'Top-2 Accuracy': f"{metrics['test_top2_accuracy']:.4f}",
            'mAP': f"{metrics['mean_ap']:.4f}",
            'F1-Score': f"{metrics['f1_macro']:.4f}",
            'Precision': f"{metrics['precision_macro']:.4f}",
            'Recall': f"{metrics['recall_macro']:.4f}",
            'Parameters (M)': f"{metrics['parameters']/1e6:.2f}",
            'Inference (ms)': f"{metrics['inference_time_per_image']*1000:.1f}",
            'FPS': f"{1/(metrics['inference_time_per_image']):.1f}",
            'Efficiency Score': f"{metrics['test_accuracy']/(metrics['parameters']/1e6):.3f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.sort_values('Test Accuracy', ascending=False)
    
    return summary_df

final_summary = create_final_summary_table(results)

print("\n" + "="*140)
print("FINAL COMPREHENSIVE SUMMARY TABLE")
print("="*140)
print(final_summary.to_string(index=False))

# Save all results
final_summary.to_csv('comprehensive_model_comparison.csv', index=False)
training_summary.to_csv('training_characteristics_summary.csv', index=False)
results_df.to_csv('detailed_model_results.csv')


2025-09-14 09:16:25.901551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757841386.095504      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757841386.150243      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Class distribution:
labels
diabetic_retinopathy    1098
normal                  1074
cataract                1038
glaucoma                1007
Name: count, dtype: int64
Number of unique classes: 4
Train samples: 3373
Valid samples: 506
Test samples: 338


In [ ]:
best_overall = results_df.index[0]
efficiency_scores = results_df['test_accuracy'] / (results_df['parameters'] / 1e6)
most_efficient = efficiency_scores.idxmax()
fastest = results_df['inference_time_per_image'].idxmin()

print(f" For MAXIMUM ACCURACY: {best_overall}")
print(f"    Test Accuracy: {results_df.loc[best_overall, 'test_accuracy']:.4f}")
print(f"    Parameters: {results_df.loc[best_overall, 'parameters']/1e6:.2f}M")
print(f"    Use when: Highest performance is critical, resources are not limited")

print(f"\n For BEST EFFICIENCY: {most_efficient}")
print(f"   → Efficiency Score: {efficiency_scores[most_efficient]:.4f}")
print(f"   → Test Accuracy: {results_df.loc[most_efficient, 'test_accuracy']:.4f}")
print(f"   → Use when: Balance between performance and model size matters")

print(f"\n🚀 For FASTEST INFERENCE: {fastest}")
print(f"   → Inference Time: {results_df.loc[fastest, 'inference_time_per_image']*1000:.1f} ms")
print(f"   → Test Accuracy: {results_df.loc[fastest, 'test_accuracy']:.4f}")
print(f"   → Use when: Real-time inference is required")

